In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

Downloading...
From (original): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf
From (redirected): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf&confirm=t&uuid=1cd5948f-9c7f-4aae-9639-943030dd42a4
To: /content/mpsis_setup.py
100%|██████████| 12.1k/12.1k [00:00<00:00, 20.0MB/s]


MessageError: Error: credential propagation was unsuccessful

In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={api_key}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance_text = element['distance']['text']
            distance = float(distance_text.replace(' km', '').replace(',', ''))

            duration_text = element['duration']['text']
            duration_parts = duration_text.split()
            duration = int(duration_parts[0])
            if "hour" in duration_text:
                duration = int(duration_parts[0]) * 60 + int(duration_parts[2])

            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def crop_matrix(data, N):
    return [row[:N] for row in data[:N]]

import numpy as np

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)  # n - (m + 1) non-zero elements
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result
def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def vrpb(number_of_clients, number_of_vehicles, cost_buffor, p, d, p_d, file_idx,problem_idx):
  try:
    Q = capacity_of_vehicle
    cost_shiffted = crop_matrix(cost_buffor,number_of_clients + 1)

    L_B = list(range(1, number_of_clients + 1))
    SOURCE = 0

    L = [index for index, value in enumerate(p) if value == 0]
    L0 = L[:]
    L0.insert(0,0)
    B = [index for index, value in enumerate(d) if value == 0]

    B0 = B[:]
    B0.insert(0,0)
    Cu = L_B

    V = list(range(0, number_of_clients + 1))

    K = number_of_vehicles
    D = p_d
    Kl = K
    Kb = K

    s = pulp.LpVariable.dicts('s', [(i, j) for i in V for j in V if i != j], cat = pulp.LpBinary)
    e = pulp.LpVariable.dicts('e', [(i, j) for i in L for j in B0 if i != j], cat = pulp.LpBinary)
    l = pulp.LpVariable.dicts('l', [(i, j) for i in V for j in V if i != j], 0, cat = "Continuous")

    model = pulp.LpProblem("vehicle_routing", pulp.LpMinimize)

    cost_function_model = pulp.lpSum(s[i,j] * cost_shiffted[i][j] for i in V for j in V if i != j) + pulp.lpSum(e[i,j] * cost_shiffted[i][j] for i in L for j in B0 if i != j)
    model += cost_function_model

    model += pulp.lpSum(s[i,j] for i in L0 for j in L if i != j) == len(L)

    for j in L:
      model += pulp.lpSum(l[i,j] for i in L0 if i != j) - pulp.lpSum(l[j,k] for k in L0 if k != j) == D[j]

    for j in L:
      model += pulp.lpSum(s[i,j] for i in L0 if i != j) == 1

    for j in L:
      model += pulp.lpSum(s[j,k] for k in L if k != j) + pulp.lpSum(e[j,k] for k in B0 if k != j)  ==  pulp.lpSum(s[(i,j)] for i in L0 if i != j)

    for j in L:
      for i in L0:
        if i != j:
          model += l[(i,j)] <= Q * s[(i,j)]

    fraaction7 = pulp.lpSum(D[j] for j in L)/Q
    model += pulp.lpSum(s[0,j] for j in L) >= max(float(fraaction7.value()),Kl)

    model += pulp.lpSum(s[i,j] for i in B for j in B0 if i != j) == len(B)

    for j in B:
      model += pulp.lpSum(l[i,j] for i in B if i != j) - pulp.lpSum(l[(j,k)] for k in B0 if k != j) == -D[j]

    for j in B:
      model += pulp.lpSum(s[j,i] for i in B0 if j != i) == 1

    for j in B:
      model += pulp.lpSum(s[k,j] for k in B if k != j) + pulp.lpSum(e[k,j] for k in L) == pulp.lpSum(s[(j,i)] for i in B0 if j != i)

    for j in B0:
      for i in B:
        if i != j:
          model += l[i,j] <= Q * s[i,j]

    fraaction13 = pulp.lpSum(D[j] for j in B)/Q
    model += max(int(fraaction13.value()),Kb) <= pulp.lpSum(s[i,0] for i in B)
    model += pulp.lpSum(s[(i,0)] for i in B) <= pulp.lpSum(s[0, j] for j in L)

    model += pulp.lpSum(s[i,0] for i in B) + pulp.lpSum(e[i, 0] for i in L) == pulp.lpSum(s[0,j] for j in L)

    for i in V:
      for j in V:
        if i != j:
          model += s[i,j] + s[j,i] <= 1

    model += pulp.lpSum(s[i,j] for i in B for j in L if i != j) == 0

    model += pulp.lpSum(s[0, j] for j in B) == 0

    model += pulp.lpSum(s[i,j] for i in L for j in B0 if i != j) == 0

    model.solve()
    if model.status != 1:
      print("INFEISABLE")
      values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))
      return None, None, None
    else:
      all_uesd_edges = []

      for i in V:
        for j in V:
          if i != j:
            if  s[i,j].varValue == 1:
              all_uesd_edges.append((i,j))

      for i in L:
        for j in B0:
          if i != j:
            if  e[i,j].varValue == 1:
              all_uesd_edges.append((i,j))

    result_paths = find_paths_vrb(all_uesd_edges, number_of_vehicles)

    return pulp.value(model.objective), result_paths, all_uesd_edges

  except Exception as e:
      print(f"error for delivery stops {number_of_clients} and vehicles {number_of_vehicles}")

      values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))

      return None, None, None

In [ ]:
def vrpb_error(number_of_clients, number_of_vehicles, cost_buffor, p, d, p_d, file_idx):
  try:
    Q = capacity_of_vehicle
    cost_shiffted = crop_matrix(cost_buffor,number_of_clients + 1)

    L_B = list(range(1, number_of_clients + 1))
    SOURCE = 0

    L = [index for index, value in enumerate(p) if value == 0]
    L0 = L[:]
    L0.insert(0,0)
    B = [index for index, value in enumerate(d) if value == 0]

    B0 = B[:]
    B0.insert(0,0)
    Cu = L_B

    V = list(range(0, number_of_clients + 1))

    K = number_of_vehicles
    D = p_d
    Kl = K
    Kb = K

    s = pulp.LpVariable.dicts('s', [(i, j) for i in V for j in V if i != j], cat = pulp.LpBinary)
    e = pulp.LpVariable.dicts('e', [(i, j) for i in L for j in B0 if i != j], cat = pulp.LpBinary)
    l = pulp.LpVariable.dicts('l', [(i, j) for i in V for j in V if i != j], 0, cat = "Continuous")

    model = pulp.LpProblem("vehicle_routing", pulp.LpMinimize)

    cost_function_model = pulp.lpSum(s[i,j] * cost_shiffted[i][j] for i in V for j in V if i != j) + pulp.lpSum(e[i,j] * cost_shiffted[i][j] for i in L for j in B0 if i != j)
    model += cost_function_model

    model += pulp.lpSum(s[i,j] for i in L0 for j in L if i != j) == len(L)

    for j in L:
      model += pulp.lpSum(l[i,j] for i in L0 if i != j) - pulp.lpSum(l[j,k] for k in L0 if k != j) == D[j]

    for j in L:
      model += pulp.lpSum(s[i,j] for i in L0 if i != j) == 1

    for j in L:
      model += pulp.lpSum(s[j,k] for k in L if k != j) + pulp.lpSum(e[j,k] for k in B0 if k != j)  ==  pulp.lpSum(s[(i,j)] for i in L0 if i != j)

    for j in L:
      for i in L0:
        if i != j:
          model += l[(i,j)] <= Q * s[(i,j)]

    fraaction7 = pulp.lpSum(D[j] for j in L)/Q
    model += pulp.lpSum(s[0,j] for j in L) >= max(float(fraaction7.value()),Kl)

    model += pulp.lpSum(s[i,j] for i in B for j in B0 if i != j) == len(B)

    for j in B:
      model += pulp.lpSum(l[i,j] for i in B if i != j) - pulp.lpSum(l[(j,k)] for k in B0 if k != j) == -D[j]

    for j in B:
      model += pulp.lpSum(s[j,i] for i in B0 if j != i) == 1

    for j in B:
      model += pulp.lpSum(s[k,j] for k in B if k != j) + pulp.lpSum(e[k,j] for k in L) == pulp.lpSum(s[(j,i)] for i in B0 if j != i)

    for j in B0:
      for i in B:
        if i != j:
          model += l[i,j] <= Q * s[i,j]

    fraaction13 = pulp.lpSum(D[j] for j in B)/Q
    model += max(int(fraaction13.value()),Kb) <= pulp.lpSum(s[i,0] for i in B)
    model += pulp.lpSum(s[(i,0)] for i in B) <= pulp.lpSum(s[0, j] for j in L)

    model += pulp.lpSum(s[i,0] for i in B) + pulp.lpSum(e[i, 0] for i in L) == pulp.lpSum(s[0,j] for j in L)

    for i in V:
      for j in V:
        if i != j:
          model += s[i,j] + s[j,i] <= 1

    model += pulp.lpSum(s[i,j] for i in B for j in L if i != j) == 0

    model += pulp.lpSum(s[0, j] for j in B) == 0

    model += pulp.lpSum(s[i,j] for i in L for j in B0 if i != j) == 0

    model.solve()

    if model.status != 1:
      print("INFEISABLE")
      return None, None, None
    else:
      all_uesd_edges = []

      for i in V:
        for j in V:
          if i != j:
            if  s[i,j].varValue == 1:
              all_uesd_edges.append((i,j))

      for i in L:
        for j in B0:
          if i != j:
            if  e[i,j].varValue == 1:
              all_uesd_edges.append((i,j))

      result_paths = find_paths_vrb(all_uesd_edges, number_of_vehicles)

      return pulp.value(model.objective), result_paths, all_uesd_edges

  except Exception as e:
    print(f"ONE MORE TIME ERROR")
    return None, None, None

In [ ]:
def create_cost_matrixes(df):
    real_distance_columns = [col for col in df.columns if col.startswith('Road Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_raod = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Road Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_raod[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Time to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_time = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Time to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_time[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Real Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_real = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Real Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_real[i][j] = float(distance)

    return cost_raod, cost_time, cost_real

### CRACOW

In [ ]:
# files = ['30_clients_cracow_1', '30_clients_cracow_2']#, '30_clients_cracow_3.csv','30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']

['30_clients_cracow_1', '30_clients_cracow_2', '30_clients_cracow_3.csv', '30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']


In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(2)]
all_paths_road = [[[] for _ in range(3)] for _ in range(2)]
all_paths_time = [[[] for _ in range(3)] for _ in range(2)]
all_paths_real = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(2)]

values_to_reload = []
vechicles = [1,2,3]
clients = [20,30]
backhaul = 0

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      if number_of_clients == 10:
        backhaul = 4
      else:
        backhaul = 7

      d = generate_demand_list(number_of_clients + 1, backhaul, 0.7 * capacity_of_vehicle * number_of_vehicles)
      p_d = replace_zeros_with_sum(d, 0.7 * capacity_of_vehicle * number_of_vehicles)
      p = [-1]

      for iterator in range(1,len(d)):
        if d[iterator] == 0:
          p.append(p_d[iterator])
        else:
          p.append(0)

      obj_road, paths_road, edges_road = vrpb(number_of_clients, number_of_vehicles, cost_R, p, d, p_d, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = vrpb(number_of_clients, number_of_vehicles, cost_T, p, d, p_d, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = vrpb(number_of_clients, number_of_vehicles, cost_G, p, d, p_d, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_cracow_1
############# calculated_file 30_clients_cracow_2
############# calculated_file 30_clients_cracow_3.csv
############# calculated_file 30_clients_cracow_4.csv
############# calculated_file 30_clients_cracow_5.csv
############# calculated_file 30_clients_cracow_6.csv
############# calculated_file 30_clients_cracow_7.csv
############# calculated_file 30_clients_cracow_8.csv
############# calculated_file 30_clients_cracow_9.csv
############# calculated_file 30_clients_cracow_10.csv
############# calculated_file 30_clients_cracow_11.csv
############# calculated_file 30_clients_cracow_12.csv
############# calculated_file 30_clients_cracow_13.csv
############# calculated_file 30_clients_cracow_14.csv
############# calculated_file 30_clients_cracow_15.csv
############# calculated_file 30_clients_cracow_16.csv
############# calculated_file 30_clients_cracow_17.csv
############# calculated_file 30_clients_cracow_18.csv
############# calculated_fi

In [ ]:
percent = 0.7

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)

    if number_of_clients == 10:
        backhaul = 4
    else:
        backhaul = 7

    d = generate_demand_list(number_of_clients + 1, backhaul, percent * capacity_of_vehicle * number_of_vehicles)
    p_d = replace_zeros_with_sum(d, percent * capacity_of_vehicle * number_of_vehicles)
    p = [-1]

    for iterator in range(1,len(d)):
      if d[iterator] == 0:
        p.append(p_d[iterator])
      else:
        p.append(0)


    obj_1, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_road, p, d, p_d, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_time, p, d, p_d, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_real, p, d, p_d, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):
      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(2)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_G = [[[] for _ in range(3)] for _ in range(2)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_T = [[[] for _ in range(3)] for _ in range(2)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_edges = [[[] for _ in range(3)] for _ in range(2)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_G_edges = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

G_T_edges = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_paths = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths = [[[] for _ in range(3)] for _ in range(2)]


for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")


AVG_R = [[69.46485000000001, 70.15985, 74.25985], [81.1791, 82.35225, 83.6092]]
AVG_R_stdev = [[6.602204020438929, 5.45690309859906, 7.240327439246102], [6.648911030386856, 6.646458793034078, 6.982876904256581]]
AVG_T = [[160.5, 165.1, 175.25], [193.0, 196.65, 203.1]]
AVG_T_stdev = [[15.577226967595998, 13.11830781770271, 15.019570566430986], [16.133815419794537, 15.097267964767665, 16.588851678160246]]
AVG_G = [[30.516000000000002, 31.074499999999993, 33.3065], [35.1725, 36.31249999999999, 37.5595]]
AVG_G_stdev = [[2.2439549906359546, 2.5481100349082273, 3.3918641998169714], [2.824690912294653, 3.09879149831027, 2.487401204068215]]
R_T_edges = [[0.543331049918512, 0.6511330698287221, 0.6892171600370027], [0.5952974836543385, 0.6146321386946387, 0.6817383669885866]]
R_T_edges_stdev = [[0.19551523340261068, 0.19714079152517336, 0.15956209679266528], [0.1978608757354282, 0.1511722494078044, 0.13956788561927982]]
R_G_edges = [[0.4980534435792272, 0.4719916556873079, 0.506859004008634], [0

### EUROPE

In [ ]:
files = ['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4','30_clients_europe_5', '30_clients_europe_6','30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4','30_clients_europe_5', '30_clients_europe_6','30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4','30_clients_europe_5', '30_clients_europe_6','30_clients_europe_5', '30_clients_europe_6']

['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4', '30_clients_europe_5', '30_clients_europe_8', '30_clients_europe_7', '30_clients_europe_6', '30_clients_europe_9', '30_clients_europe_10', '30_clients_europe_11', '30_clients_europe_12', '30_clients_europe_13', '30_clients_europe_14', '30_clients_europe_15', '30_clients_europe_16', '30_clients_europe_17', '30_clients_europe_18', '30_clients_europe_19', '30_clients_europe_20']


20

In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(2)]
all_paths_road = [[[] for _ in range(3)] for _ in range(2)]
all_paths_time = [[[] for _ in range(3)] for _ in range(2)]
all_paths_real = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(2)]

values_to_reload = []
vechicles = [1,2,3]
clients = [10,30]
backhaul = 0

for requested_file in range(len(files)):
  print(f"############# calculated_file {requested_file}")
  df = pd.read_csv(os.path.join(output_dir, files[requested_file]))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      if number_of_clients == 10:
        backhaul = 4
      else:
        backhaul = 7

      d = generate_demand_list(number_of_clients + 1, backhaul, 0.7 * capacity_of_vehicle * number_of_vehicles)
      p_d = replace_zeros_with_sum(d, 0.7 * capacity_of_vehicle * number_of_vehicles)
      p = [-1]

      for iterator in range(1,len(d)):
        if d[iterator] == 0:
          p.append(p_d[iterator])
        else:
          p.append(0)

      obj_road, paths_road, edges_road = vrpb(number_of_clients, number_of_vehicles, cost_R, p, d, p_d, requested_file, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = vrpb(number_of_clients, number_of_vehicles, cost_T, p, d, p_d, requested_file, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = vrpb(number_of_clients, number_of_vehicles, cost_G, p, d, p_d, requested_file, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

In [ ]:
percent = 0.7

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)

    if number_of_clients == 10:
        backhaul = 4
    else:
        backhaul = 7

    d = generate_demand_list(number_of_clients + 1, backhaul, percent * capacity_of_vehicle * number_of_vehicles)
    p_d = replace_zeros_with_sum(d, percent * capacity_of_vehicle * number_of_vehicles)
    p = [-1]

    for iterator in range(1,len(d)):
      if d[iterator] == 0:
        p.append(p_d[iterator])
      else:
        p.append(0)


    obj_1, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_road, p, d, p_d, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_time, p, d, p_d, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = vrpb_error(number_of_clients, number_of_vehicles, cost_real, p, d, p_d, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
print(all_uesd_edges_time)

[[[[(0, 1), (0, 5), (1, 9), (2, 0), (3, 7), (4, 10), (5, 8), (6, 0), (7, 6), (9, 4), (8, 3), (10, 2)], [(0, 4), (0, 9), (1, 0), (2, 0), (4, 5), (5, 10), (6, 1), (7, 6), (9, 8), (10, 3), (3, 2), (8, 7)], [(0, 7), (1, 4), (2, 0), (3, 5), (4, 8), (6, 10), (7, 1), (8, 3), (9, 2), (10, 9), (5, 6)], [(0, 2), (0, 9), (3, 0), (4, 6), (5, 1), (6, 8), (7, 10), (8, 0), (9, 7), (10, 5), (1, 3), (2, 4)], [(0, 3), (1, 7), (2, 10), (3, 9), (4, 5), (5, 0), (7, 8), (8, 6), (9, 1), (10, 4), (6, 2)], [(0, 8), (1, 7), (2, 5), (3, 9), (4, 10), (5, 6), (7, 4), (8, 3), (9, 2), (10, 0), (6, 1)], [(0, 6), (0, 9), (1, 0), (2, 10), (4, 1), (5, 3), (6, 5), (7, 0), (8, 7), (9, 2), (3, 8), (10, 4)], [(0, 1), (0, 2), (1, 9), (2, 3), (4, 0), (5, 4), (6, 0), (8, 7), (9, 8), (10, 5), (3, 10), (7, 6)], [(0, 5), (1, 4), (2, 0), (3, 6), (5, 10), (6, 2), (7, 1), (8, 3), (9, 7), (10, 9), (4, 8)], [(0, 6), (0, 9), (1, 8), (2, 0), (3, 5), (4, 2), (7, 10), (8, 0), (9, 7), (10, 3), (5, 1), (6, 4)], [(0, 4), (1, 3), (2, 1), (3, 

In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):
      print(f"{file_idx}_{vechicles_idx}_{clients_idx}")
      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))




In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(2)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_G = [[[] for _ in range(3)] for _ in range(2)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_T = [[[] for _ in range(3)] for _ in range(2)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_edges = [[[] for _ in range(3)] for _ in range(2)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_G_edges = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

G_T_edges = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_paths = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths = [[[] for _ in range(3)] for _ in range(2)]


for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")



AVG_R = [[3912.4449999999997, 4191.035000000002, 4840.839999999999], [6639.219999999999, 6737.779999999999, 7112.045000000001]]
AVG_R_stdev = [[422.63635252424746, 473.4835269309799, 372.0785110699086], [476.8395103176749, 466.3699911014859, 495.43443004599504]]
AVG_T = [[2640.5, 2824.15, 3240.25], [4927.5, 4997.05, 5239.4]]
AVG_T_stdev = [[284.27952792981773, 300.9282763383993, 245.78585699750911], [306.6065067802704, 293.4836409410242, 267.9407397168262]]
AVG_G = [[2857.6960000000004, 3085.4649999999997, 3593.002500000001], [4852.764, 4919.9400000000005, 5203.573999999999]]
AVG_G_stdev = [[280.78299728437986, 343.5830576367234, 300.67392156412575], [345.61862757091063, 344.0530822562121, 367.47273152439453]]
R_T_edges = [[0.9128787878787878, 0.8578333333333333, 0.9007122507122508], [0.6069177104257749, 0.7639940268065268, 0.7193783755021684]]
R_T_edges_stdev = [[0.2133957919071346, 0.22964047988105235, 0.1510570085088551], [0.24261596494884713, 0.15728529664004628, 0.1849709515575358

### RESULTS

In [ ]:
for client in clients:
  for vechicle in vechicles:
    df.loc[iterator] = [topology, client, vechicle,
                        str(AVG_R[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_R_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_G[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_G_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_T[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_T_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(R_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(R_G_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_G_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(G_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(G_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        R_T_paths[clients.index(client)][vechicles.index(vechicle)], R_G_paths[clients.index(client)][vechicles.index(vechicle)], G_T_paths[clients.index(client)][vechicles.index(vechicle)]
                        ]
    iterator += 1


In [ ]:
| Topology | No. clients | No. vehicles | Road distance matrix | Geographical distance matrix | Time distance matrix |
|----------|-------------|--------------|----------------------|-----------------------------|----------------------|
| Krakow   | 10          | 1            | 47.0 ± 4.852         | 22.207 ± 1.25               | 105.5 ± 9.031        |
| Krakow   | 10          | 2            | 49.255 ± 5.502       | 23.37 ± 1.486               | 112.85 ± 10.578      |
| Krakow   | 10          | 3            | 55.755 ± 5.378       | 27.07 ± 1.384               | 129.0 ± 10.104       |
| Krakow   | 15          | 1            | 62.764 ± 4.234       | 27.792 ± 1.205              | 143.25 ± 8.727       |
| Krakow   | 15          | 2            | 63.459 ± 4.047       | 27.861 ± 1.456              | 145.45 ± 8.112       |
| Krakow   | 15          | 3            | 67.095 ± 4.104       | 30.032 ± 1.912              | 155.6 ± 8.91         |
| Krakow   | 20          | 1            | 69.465 ± 3.09        | 30.516 ± 1.05               | 160.5 ± 7.29         |
| Krakow   | 20          | 2            | 70.16 ± 2.554        | 31.074 ± 1.193              | 165.1 ± 6.139        |
| Krakow   | 20          | 3            | 74.26 ± 3.388        | 33.307 ± 1.588              | 175.25 ± 7.03        |
| Krakow   | 30          | 1            | 81.179 ± 3.112       | 35.173 ± 1.322              | 193.0 ± 7.551        |
| Krakow   | 30          | 2            | 82.352 ± 3.11        | 36.312 ± 1.45               | 196.65 ± 7.066       |
| Krakow   | 30          | 3            | 83.609 ± 3.268       | 37.56 ± 1.164               | 203.4 ± 7.764        |
| Europe   | 10          | 1            | 3944.69 ± 281.381    | 2913.789 ± 196.246          | 2697.1 ± 186.356     |
| Europe   | 10          | 2            | 4314.725 ± 304.38    | 3199.94 ± 232.293           | 2930.0 ± 202.074     |
| Europe   | 10          | 3            | 4932.36 ± 320.235    | 3690.717 ± 233.064          | 3306.6 ± 206.945     |
| Europe   | 15          | 1            | 4939.825 ± 283.003   | 3661.857 ± 200.721          | 3429.55 ± 186.585    |
| Europe   | 15          | 2            | 5178.91 ± 295.161    | 3833.26 ± 204.383           | 3602.0 ± 183.176     |
| Europe   | 15          | 3            | 5738.605 ± 317.968   | 4247.262 ± 227.423          | 3934.75 ± 197.613    |
| Europe   | 20          | 1            | 5728.915 ± 248.467   | 4219.703 ± 183.24           | 4053.55 ± 150.22     |
| Europe   | 20          | 2            | 5841.485 ± 241.291   | 4290.101 ± 165.477          | 4148.35 ± 146.889    |
| Europe   | 20          | 3            | 6282.1 ± 231.037     | 4612.026 ± 161.196          | 4438.75 ± 145.681    |
| Europe   | 30          | 1            | 6639.22 ± 223.168    | 4852.764 ± 161.755          | 4927.5 ± 133.496     |
| Europe   | 30          | 2            | 6737.78 ± 218.26     | 4919.94 ± 161.022           | 4997.05 ± 137.355    |
| Europe   | 30          | 3            | 7112.045 ± 231.87    | 5203.574 ± 171.983          | 5239.4 ± 125.4       |


In [ ]:
| Topology | No. clients | No. vehicles | R/T e. [\%]       | R/G e. [\%]       | G/T e. [\%]       | R/T p. | R/G p. | G/T p. |
|----------|-------------|--------------|-------------------|-------------------|-------------------|--------|--------|--------|
| Krakow   | 10          | 1            | 54.42 ± 11.97      | 50.59 ± 9.45       | 48.59 ± 8.99       | 4      | 2      | 1      |
| Krakow   | 10          | 2            | 71.67 ± 9.93       | 61.01 ± 9.20       | 63.21 ± 9.34       | 5      | 2      | 3      |
| Krakow   | 10          | 3            | 76.96 ± 7.30       | 66.74 ± 8.49       | 67.15 ± 9.50       | 5      | 3      | 3      |
| Krakow   | 15          | 1            | 66.42 ± 11.22      | 56.69 ± 7.08       | 50.65 ± 7.84       | 4      | 1      | 0      |
| Krakow   | 15          | 2            | 67.55 ± 11.20      | 47.11 ± 10.45      | 50.19 ± 8.98       | 3      | 2      | 1      |
| Krakow   | 15          | 3            | 79.48 ± 9.04       | 57.31 ± 8.36       | 51.17 ± 8.90       | 7      | 1      | 0      |
| Krakow   | 20          | 1            | 54.33 ± 9.15       | 49.81 ± 8.34       | 38.26 ± 5.73       | 2      | 2      | 0      |
| Krakow   | 20          | 2            | 65.11 ± 9.23       | 47.20 ± 5.84       | 46.90 ± 5.82       | 2      | 0      | 0      |
| Krakow   | 20          | 3            | 68.92 ± 7.47       | 50.69 ± 7.83       | 52.90 ± 8.94       | 0      | 0      | 0      |
| Krakow   | 30          | 1            | 59.53 ± 9.26       | 44.53 ± 5.08       | 33.34 ± 6.52       | 0      | 0      | 0      |
| Krakow   | 30          | 2            | 61.46 ± 7.08       | 47.68 ± 5.50       | 43.71 ± 4.91       | 0      | 0      | 0      |
| Krakow   | 30          | 3            | 68.17 ± 6.53       | 43.41 ± 3.61       | 43.90 ± 6.13       | 0      | 0      | 0      |
| Europe   | 10          | 1            | 81.83 ± 10.65      | 81.05 ± 9.87       | 77.04 ± 9.65       | 15     | 12     | 11     |
| Europe   | 10          | 2            | 82.81 ± 8.40       | 78.74 ± 11.53      | 74.53 ± 10.26      | 16     | 13     | 12     |
| Europe   | 10          | 3            | 89.23 ± 7.59       | 87.31 ± 7.91       | 85.77 ± 9.54       | 12     | 11     | 12     |
| Europe   | 15          | 1            | 81.52 ± 10.44      | 77.93 ± 7.72       | 65.86 ± 9.98       | 9      | 8      | 5      |
| Europe   | 15          | 2            | 84.17 ± 7.63       | 85.07 ± 8.14       | 83.88 ± 7.56       | 8      | 9      | 8      |
| Europe   | 15          | 3            | 82.05 ± 8.08       | 79.92 ± 5.72       | 74.88 ± 5.04       | 7      | 3      | 1      |
| Europe   | 20          | 1            | 82.34 ± 10.03      | 77.83 ± 9.47       | 67.73 ± 10.88      | 9      | 10     | 7      |
| Europe   | 20          | 2            | 85.04 ± 6.83       | 79.85 ± 5.58       | 79.33 ± 8.27       | 8      | 2      | 4      |
| Europe   | 20          | 3            | 87.72 ± 7.02       | 85.41 ± 6.17       | 80.61 ± 7.13       | 6      | 7      | 4      |
| Europe   | 30          | 1            | 70.69 ± 11.36      | 60.60 ± 11.62      | 58.17 ± 11.01      | 5      | 2      | 2      |
| Europe   | 30          | 2            | 76.40 ± 7.36       | 73.62 ± 8.16       | 71.71 ± 8.96       | 3      | 3      | 3      |
| Europe   | 30          | 3            | 79.94 ± 8.66       | 71.29 ± 5.85       | 70.55 ± 6.41       | 2      | 2      | 1      |
